## Desafío Cazatalentos

In [2]:
# Probabilidades de que gustavo sea ganador sobre michael jackson

gustavo_ganador <- 0
for( i in 1:10000 )
{
 aciertos_michael <- sum( runif(10) < 0.85 )
 aciertos_gustavo <- sum( runif(10) < 0.10 )
 if( aciertos_gustavo > aciertos_michael ) gustavo_ganador <-
gustavo_ganador +1
}
print( gustavo_ganador )

[1] 0


## Segundo Caso, aparece el overfitting

El cazatalentos llega ahora a un pueblo donde hay 100 jugadores, los que son mucho más parejos entre si desde el punto de vista del ratio de enceste.

Hay un jugador, que llamaremos jugador 1 que tiene un “índice de enceste” de 0.70.

Los 99 jugadores restantes que llamaremos “el pelotón” tienen los indices de enceste { 0.501, 0.502, 0.503, …, 0.599 }

Es decir el jugador 1 tiene 0.70 y el mejor jugador del pelotón tiene 0.599, hay un poco más de 0.10 de diferencia. La diferencia de 0.10 es significativa en basket.

¿Qué sucede con el algoritmo del cazatalentos de hacer tirar 10 tiros libres a cada uno de los jugadores y elegir al que más encestes logró ?

El resultado se puede ver en el script basket_02.r

Aquí ya pasa algo asombroso, el overfitting en todo su esplendor.

Si hago tirar 10 tiros libres a cada uno de los 100 jugadores, apenas 1450 veces de las 10000 este método devuelve al verdadero mejor, que es el jugador con un índice de enceste de 0.70.

Lo que está sucediendo es que la inmensa mayoría de las veces, uno de los 99 jugadores del pelotón tiene mucha suerte y supera al jugador de 0.70, con lo cual se elige a uno del pelotón y no al verdadero mejor ! El tener 99 jugadores en el pelotón hace que al ser tantos hay varios que tienen mala suerte, pero también hay varios con muy buena suerte, y el que tuvo más suerte supera al verdadero mejor !

In [3]:
# Script basket_02.r

require("data.table")

set.seed( 333333 )

#calcula cuantos encestes logra un jugador con indice de enceste prob que hace qyt tiros libres
ftirar  <- function( prob, qty )
{
  return( sum( runif(qty) < prob ) )
}


#defino los jugadores
mejor      <- 0.7
peloton    <- ( 501:599 ) / 1000
jugadores  <- c( mejor, peloton )

#veo que tiene el vector
jugadores

#hago que los 100 jugadores tiren 10 veces cada uno
mapply( ftirar, jugadores, 10 )

primero_ganador  <- 0

for( i in 1:10000 )  #diez mil experimentos
{
  vaciertos  <- mapply( ftirar, jugadores, 10 )  #10 tiros libres cada jugador

  mejor  <- which.max( vaciertos )
  if( mejor == 1 )  primero_ganador  <- primero_ganador + 1
}

print(  primero_ganador )

[1] 0.700 0.501 0.502 0.503 0.504 0.505 0.506 0.507 0.508 0.509 0.510 0.511
 [13] 0.512 0.513 0.514 0.515 0.516 0.517 0.518 0.519 0.520 0.521 0.522 0.523
 [25] 0.524 0.525 0.526 0.527 0.528 0.529 0.530 0.531 0.532 0.533 0.534 0.535
 [37] 0.536 0.537 0.538 0.539 0.540 0.541 0.542 0.543 0.544 0.545 0.546 0.547
 [49] 0.548 0.549 0.550 0.551 0.552 0.553 0.554 0.555 0.556 0.557 0.558 0.559
 [61] 0.560 0.561 0.562 0.563 0.564 0.565 0.566 0.567 0.568 0.569 0.570 0.571
 [73] 0.572 0.573 0.574 0.575 0.576 0.577 0.578 0.579 0.580 0.581 0.582 0.583
 [85] 0.584 0.585 0.586 0.587 0.588 0.589 0.590 0.591 0.592 0.593 0.594 0.595
 [97] 0.596 0.597 0.598 0.599

[1] 4 2 7 8 4 5 5 7 4 8 6 7 5 5 5 5 8 2 5 6 2 6 3 6 8 7 8 4 5 5 3 4 7 5 6 6 5
 [38] 4 6 5 8 4 5 2 5 4 6 6 4 8 9 7 6 5 7 4 6 5 6 6 6 8 4 5 7 5 5 7 3 8 5 4 6 5
 [75] 5 7 5 4 7 6 5 8 7 6 4 7 5 5 4 2 6 6 6 7 4 7 8 7 7 7

[1] 1407


## 3er caso

Ahora pasemos a ver aún algo más notable.

Tenemos a nuestro jugador de 0.70, y a los 99 jugadores del pelotón. Los hacemos tirar a cada uno 10 tiros libres, elegimos al ganador registrando cuantos aciertos tuvo, y solo a ese ganador lo hacemos tirar una nueva ronda de 10 tiros libres, finalmente comparamos estos nuevos aciertos con los originales.

Este puede verse en el script basket_03.r

Lo que se observa en este caso es **La maldición del ganador**, la performance que el jugador que logró más aciertos en la competencia general luego NO LA PUEDE MANTENER, en todos los casos vemos que en la nueva ronda de diez tiene menos aciertos que en la primera ronda de diez.

La primera ronda el que resultó ganador fue debido a la suerte, y esa suerte ya no lo acompaña para la segunda ronda de 10 tiros libres.

In [5]:
# script basket_03.r

require("data.table")

set.seed( 333333 )

#calcula cuantos encestes logra un jugador con indice de enceste prob que hace qyt tiros libres
ftirar  <- function( prob, qty )
{
  return( sum( runif(qty) < prob ) )
}

mejor     <-  0.7
peloton   <-  ( 501:599 ) / 1000
jugadores <-  c( mejor, peloton )

#veo que tiene el vector
jugadores


for( i in 1:10 )
{
   vaciertos  <- mapply( ftirar, jugadores, 10 )  #cada jugador tira 10 tiros libres

   mejor  <- which.max( vaciertos )

   aciertos_torneo  <- vaciertos[ mejor ]

   aciertos_segunda  <- ftirar( jugadores[ mejor ], 10 )

   cat( aciertos_torneo, aciertos_segunda, "\n" )
}

[1] 0.700 0.501 0.502 0.503 0.504 0.505 0.506 0.507 0.508 0.509 0.510 0.511
 [13] 0.512 0.513 0.514 0.515 0.516 0.517 0.518 0.519 0.520 0.521 0.522 0.523
 [25] 0.524 0.525 0.526 0.527 0.528 0.529 0.530 0.531 0.532 0.533 0.534 0.535
 [37] 0.536 0.537 0.538 0.539 0.540 0.541 0.542 0.543 0.544 0.545 0.546 0.547
 [49] 0.548 0.549 0.550 0.551 0.552 0.553 0.554 0.555 0.556 0.557 0.558 0.559
 [61] 0.560 0.561 0.562 0.563 0.564 0.565 0.566 0.567 0.568 0.569 0.570 0.571
 [73] 0.572 0.573 0.574 0.575 0.576 0.577 0.578 0.579 0.580 0.581 0.582 0.583
 [85] 0.584 0.585 0.586 0.587 0.588 0.589 0.590 0.591 0.592 0.593 0.594 0.595
 [97] 0.596 0.597 0.598 0.599

9 5 
9 5 
9 3 
9 6 
10 7 
9 7 
10 4 
9 6 
10 7 
9 7 


## Tercer Caso, el overfitting en su plenitud
¿Cuándo es más caso extremo el efecto de “La maldición del ganador” ?

Supongamos por un momento que ahora tenemos 100 jugadores, todos con un “índice de enceste” de 0.70 ; recordar que ese valor jamás es conocido por el cazatalentos.

Ahora nos ponemos más estrictos, y los hacemos tirar 100 veces a cada uno, elegimos al ganador, y solo a ese ganador lo hacemos tirar 100 nuevos tiros libres. Podemos pensarlo como que el cazatalentos se lleva al mejor jugador a la gran ciudad, le habla maravillas de él al entrenador, y el entrenador dice “probémoslo a ver si es tan bueno como decís, que haga 100 tiros libres”.

In [6]:
# script basket_04.r

require("data.table")

set.seed( 333333 )

#calcula cuantos encestes logra un jugador con indice de enceste prob que hace qyt tiros libres
ftirar  <- function( prob, qty )
{
  return( sum( runif(qty) < prob ) )
}


jugadores  <- rep( 0.7, 100 )  #jugadores identicos


for( i in 1:10 )
{
   vaciertos  <- mapply( ftirar, jugadores, 100 )  #cada jugador tira 100 tiros libres

   mejor  <- which.max( vaciertos )

   aciertos_torneo  <- vaciertos[ mejor ]

   aciertos_segunda  <- ftirar( jugadores[ mejor ], 100 )

   cat( aciertos_torneo, aciertos_segunda, "\n" )
}

82 68 
82 78 
83 78 
79 64 
82 69 
81 73 
78 71 
80 69 
82 61 
80 73 


Nuevamente vemos, que el puntaje alcanzado por el mejor jugador cuando compitió contra los otros 99, NO ES VUELTO A ALCANZAR en la nueva ronda de 100 tiros libres que hace ese jugador.

Este efecto es exactamente el mismo que vemos cuando alguien se empecina en lograr en el Leaderboard Público el mayor puntaje de la clase, le va muy bien en el Público, pero se derrumba en el Privado. Decimos que “overfitió el Leaderboard Publico”.

Ahora nos preguntamos, en este caso, en promedio, cuanto más aciertos tiene el ganador (debido a la suerte) en la primera ronda con respecto a la segunda?

Nuevamente vemos, que el puntaje alcanzado por el mejor jugador cuando compitió contra los otros 99, NO ES VUELTO A ALCANZAR en la nueva ronda de 100 tiros libres que hace ese jugador.

Este efecto es exactamente el mismo que vemos cuando alguien se empecina en lograr en el Leaderboard Público el mayor puntaje de la clase, le va muy bien en el Público, pero se derrumba en el Privado. Decimos que “overfitió el Leaderboard Publico”.

Ahora nos preguntamos, en este caso, en promedio, cuanto más aciertos tiene el ganador (debido a la suerte) en la primera ronda con respecto a la segunda?

Corriendo basket_05.r vemos que la diferencia promedio es de 11.06131

Exactamente este mismo efecto se da en el marketing digital. Si en un A/B testing se prueban cien alternativas y se elige a la que tiene más efectividad, se observará que en un nuevo experimento disminuirá el rendimiento de esa alternativa.

También sucede si se prueban muchos medicamentos al mismo tiempo, el que resulte más efectivo disminuirá su efectividad en la nueva prueba.

Este problema es llamado **El problema de las múltiples comparaciones**.

In [8]:
# script basket_05.r

require("data.table")

set.seed( 333333 )

#calcula cuantos encestes logra un jugador con indice de enceste prob que hace qyt tiros libres
ftirar  <- function( prob, qty )
{
  return( sum( runif(qty) < prob ) )
}


jugadores  <- rep( 0.7, 100 )  #jugadores identicos

diferencias  <- c()

for( i in 1:9999 )
{
   vaciertos  <- mapply( ftirar, jugadores, 100 )  #cada jugador tira 100 tiros libres

   mejor  <- which.max( vaciertos )

   aciertos_torneo  <- vaciertos[ mejor ]

   aciertos_segunda  <- ftirar( jugadores[mejor], 100 )

   diferencias  <- c( diferencias, aciertos_torneo - aciertos_segunda )
}


mean( diferencias )

[1] 11.09401

## Cuarto caso, ¿cuándo se atenúa/desaparece el overfitting?

Supongamos 99 jugadores con un índice de enceste de 0.60 y ahora sumamos al adolescente Michael Jordan con su superlativo índice de enceste del 0.85.

Los hacemos tirar a todos 100 tiros libres, llevamos al ganador a la gran ciudad, y allí le contamos al entrenador de nuestro ganador Jordan. 

En este caso observamos dos cosas, en primer lugar SIEMPRE el ganador del torneo fue Michael Jordan, y en segundo cuando tuvo que mostrar en la ciudad su performance, fue la misma que en el primer torneo. Es tan bueno Jordan que le ganó al que tuvo más suerte del pelotón, el efecto suerte no está afectando a Jordan porque está solo, y gana por su superioridad, no por la suerte.

In [9]:
# script basket_06.r

require("data.table")

set.seed( 102191 )

#calcula cuantos encestes logra un jugador con indice de enceste prob que hace qyt tiros libres
ftirar  <- function( prob, qty )
{
  return( sum( runif(qty) < prob ) )
}

jordan   <- 0.85
peloton  <- rep( 0.6, 99 )  #jugadores identicos

jugadores  <- c( jordan, peloton )

for( i in 1:10 )
{
   vaciertos  <- mapply( ftirar, jugadores, 100 )  #cada jugador tira 100 tiros libres

   mejor  <- which.max( vaciertos )

   aciertos_torneo  <- vaciertos[ mejor ]

   aciertos_segunda  <- ftirar( jugadores[ mejor ], 100 )

   cat( mejor, aciertos_torneo, aciertos_segunda, "\n" )
}

1 88 91 
1 85 86 
1 79 91 
1 86 81 
1 87 87 
1 84 84 
1 86 83 
1 87 86 
1 85 77 
1 92 83 


## Quinto caso, tamaño de la ronda

El cazatalentos llega ahora a un pueblo donde hay 100 jugadores, los que son mucho más parejos entre si desde el punto de vista del ratio de enceste.

Hay un jugador, que llamaremos jugador 1 que tiene un “índice de enceste” de 0.70 , es un jugador bueno pero dista mucho de ser Michael Jordan, jamás estará en el "NBA Hall of Fame".

Los 99 jugadores restantes que llamaremos “el pelotón” tienen los índices de enceste { 0.501, 0.0502, 0.503, …, 0.599 }

Es decir el jugador 1 tiene 0.70 y el mejor jugador del pelotón tiene 0.599, o sea hay un poco más de 0.10 de diferencia. La diferencia de 0.10 es significativa en basket.

¿Cuántos tiros libres debe el cazatalentos pedirles que haga cada jugador para que si elige el mejor jugador tiene la certeza de llevarse el “verdadero mejor” el 99% de los casos ?

Corriendo el basket_07.r vemos que haciendo tirar 415 tiros libres a cada uno de los 100 jugadores, y quedándose con el ganador de ese torneo, en el 99.10% ( 0.9910 ) de los casos estoy seguro que ese es el "verdadero mejor".

In [10]:
# script basket_07.r

#intencionalmente el mejor jugador va al final de la lista de jugadores
#porque la funcion which.max() de R hace trampa
#si hay un empate ( dos máximos) se queda con el que esta primero en el vector

require("data.table")

set.seed( 333333 )

#calcula cuantos encestes logra un jugador con indice de enceste prob que hace qyt tiros libres
ftirar  <- function( prob, qty )
{
  return( sum( runif(qty) < prob ) )
}

#defino los jugadores
mejor      <-  0.7
peloton    <-  ( 501:599 ) / 1000
jugadores  <-  c( peloton, mejor ) #intencionalmente el mejor esta al final

#veo que tiene el vector
jugadores

#hago que los 100 jugadores tiren 10 veces cada uno
mapply(  ftirar, jugadores, 10 )


for(  tiros_libres  in c(10, 20, 50, 100, 200, 300, 400, 415, 500, 600, 700, 1000 ) )
{

  primero_ganador  <- 0

  for( i in 1:10000 )  #diez mil experimentos
  {
    vaciertos  <- mapply( ftirar, jugadores, tiros_libres )
    mejor  <- which.max( vaciertos )

    if( mejor == 100 )  primero_ganador  <- primero_ganador + 1
  }

  cat( tiros_libres, primero_ganador/10000, "\n" )

}


[1] 0.501 0.502 0.503 0.504 0.505 0.506 0.507 0.508 0.509 0.510 0.511 0.512
 [13] 0.513 0.514 0.515 0.516 0.517 0.518 0.519 0.520 0.521 0.522 0.523 0.524
 [25] 0.525 0.526 0.527 0.528 0.529 0.530 0.531 0.532 0.533 0.534 0.535 0.536
 [37] 0.537 0.538 0.539 0.540 0.541 0.542 0.543 0.544 0.545 0.546 0.547 0.548
 [49] 0.549 0.550 0.551 0.552 0.553 0.554 0.555 0.556 0.557 0.558 0.559 0.560
 [61] 0.561 0.562 0.563 0.564 0.565 0.566 0.567 0.568 0.569 0.570 0.571 0.572
 [73] 0.573 0.574 0.575 0.576 0.577 0.578 0.579 0.580 0.581 0.582 0.583 0.584
 [85] 0.585 0.586 0.587 0.588 0.589 0.590 0.591 0.592 0.593 0.594 0.595 0.596
 [97] 0.597 0.598 0.599 0.700

[1] 3 2 7 8 4 5 5 7 4 8 6 7 5 5 5 5 8 2 5 6 2 6 3 6 8 7 8 4 5 5 3 4 7 5 6 6 5
 [38] 4 6 5 8 4 5 2 5 4 6 6 4 8 9 7 6 5 7 4 6 5 6 6 6 8 4 5 7 5 5 7 3 8 5 4 6 5
 [75] 5 7 5 4 7 6 5 8 7 6 4 7 5 5 4 3 6 6 6 7 4 7 8 7 7 8

10 0.0336 
20 0.0854 
50 0.2692 
100 0.5495 
200 0.8573 
300 0.958 
400 0.988 
415 0.9898 
500 0.9969 
600 0.9991 
700 0.9998 
1000 1 


## Intento A

In [12]:
#Intento de Solucion del desafio  15k
#que NO logra solucionarlo, una que falta una idea fundamental, una chispa, un Momento Eureka
#pero crea estructura sobre la cual trabajar

#limpio la memoria
rm( list=ls() )
gc()

require("data.table")

ftirar  <- function( prob, qty )
{
  return(  sum( runif(qty) < prob ) )
}


# variables globales que usan las funciones gimnasio_xxxx
GLOBAL_jugadores  <- c()
GLOBAL_tiros_total  <- 0

# Crea el juego
# a cada jugador se le pone un numero de 1 a 100 en la espalda
# debajo de ese numero esta el indice_de_enceste  que NO puede ser visto por el cazatalentos

gimnasio_init  <- function() 
{
  GLOBAL_jugadores  <<-  sample( c( (501:599 ) / 1000 , 0.7 ) )
  GLOBAL_tiros_total  <<- 0
}


# se le pasa un vector con los IDs de los jugadores y la cantidad de tiros a realizar
# devuelve en un vector cuantos aciertos tuvo cada jugador

gimnasio_tirar  <- function(  pids,  pcantidad )
{
  GLOBAL_tiros_total  <<-  GLOBAL_tiros_total + length( pids )*pcantidad
  res  <- mapply(  ftirar, GLOBAL_jugadores[pids], pcantidad )

  return( res )
}


#El cazatalentos decide a que jugador llevarse
#devuelve la cantidad de tiros libres y si le acerto al verdadero_mejor o no
gimnasio_veredicto  <- function( pid )
{
  return( list("tiros_total"= GLOBAL_tiros_total, 
               "acierto"=     as.integer( GLOBAL_jugadores[pid]==0.7) ))
}
#------------------------------------------------------------------------------


#Estrategia
#En la primer ronda se hace tirar 90 tiros libres a cada uno de los 100 jugadores ( se gastan 90000 tiros )
#Se eligen a la mejor mitad  ( se descarta a la peor mitad )
#En la segunda ronda, a la mejor mitad de la anterior ronda se los hace tirar 400 tiros a cada uno
#Se elige el mejor jugador de la segunda ronda

set.seed( 102191 )

#inicializo el juego
gimnasio_init()

#Esta el la planilla del cazatalentos
planilla_cazatalentos  <- data.table( "id" = 1:100 )


#Ronda 1  ------------------------------------------------------
#tiran los 100 jugadores es decir 1:100  90 tiros libres cada uno
ids_juegan1  <- 1:100   #los jugadores que participan en la ronda,

planilla_cazatalentos[ ids_juegan1,  tiros1 := 90 ]  #registro en la planilla que tiran 90 tiros

#Hago que tiren
resultado1  <- gimnasio_tirar( ids_juegan1, 90)
planilla_cazatalentos[ ids_juegan1,  aciertos1 := resultado1 ]  #registro en la planilla


#Ronda 2 -------------------------------------------------------
#los mejores 40 jugadores tiran 400 tiros cada uno
mediana  <- planilla_cazatalentos[ ids_juegan1, median(aciertos1) ]
ids_juegan2  <- planilla_cazatalentos[ ids_juegan1 ][aciertos1 >= mediana, id ]

planilla_cazatalentos[ ids_juegan2,  tiros2 := 400 ]  #registro en la planilla que tiran 400 tiros
resultado2  <- gimnasio_tirar( ids_juegan2, 400)
planilla_cazatalentos[ ids_juegan2,  aciertos2 := resultado2 ]  #registro en la planilla

#El cazatalentos toma una decision, elige al que mas aciertos tuvo en la ronda2
pos_mejor <-  planilla_cazatalentos[ , which.max(aciertos2) ]
id_mejor  <-  planilla_cazatalentos[ pos_mejor, id ]

#Finalmente, la hora de la verdadero_mejor
#Termino el juego
veredicto  <- gimnasio_veredicto( id_mejor )

veredicto


#El veredicto da que la estrategia seguida por el cazatalentos fue exitosa para este caso
#Le acerto al verdadero_mejor

#En el siguiente script veremos de hacer una Estimacion Montecarlo
#De 10000 veces que el entrenador sigue esta estrategia, cuantas realmente le acierta

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,536467,28.7,1197860,64,1197860,64
Vcells,998221,7.7,8388608,64,8388608,64


$tiros_total
[1] 29000

$acierto
[1] 1